In [1]:
import pandas as pd
import numpy as np

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [22]:
def load_unigram_data():
    #df = pd.read_csv('~/Downloads/ias-officers/analysis/processed/unigram maps.csv')
    df = pd.read_csv('./processed/unigram maps.csv')
    #Dropping rows where the experience is NA
    df = df.loc[df["Experience"] != "N.A."]
    
    #Adding a new column that contains the count value as string, for example 'size:count' 
    df["text"] = df["Count"].apply(lambda x: "size: "+str(x))
    
    #Adding a percentage column per subject
    df["p_Subject"] = [0] * df.shape[0]
    for subject in df["Subject"].unique():
        sum_subject_count = df.loc[df["Subject"] == subject]["Count"].sum()
        df.loc[df["Subject"] == subject, "p_Subject"] = df.loc[df["Subject"] == subject]["Count"].apply(lambda x: str(round(((x / sum_subject_count) * 100), 2)) + "%")

    df["p_Experience"] = [0] * df.shape[0]
    for experience in df["Experience"].unique():
        sum_experience_count = df.loc[df["Experience"] == experience]["Count"].sum()
        df.loc[df["Experience"] == experience, "p_Experience"] = df.loc[df["Experience"] == experience]["Count"].apply(lambda x: str(round(((x / sum_experience_count) * 100), 2)) + "%")
    return df

In [20]:
df = load_unigram_data()

In [21]:
df

,Subject,Experience,Count,text,p_Subject,p_Experience
0,Social Science and Liberal Arts,Land Revenue Mgmt & District Admn,1736,size: 1736,17.03,27.88
1,"Governance, Law and Management",Land Revenue Mgmt & District Admn,1227,size: 1227,16.19,19.70
2,"Engineering, Technology and Mathematics",Land Revenue Mgmt & District Admn,1104,size: 1104,17.81,17.73
3,Social Science and Liberal Arts,Personnel and General Administration,828,size: 828,8.12,28.97
4,Natural Sciences,Land Revenue Mgmt & District Admn,815,size: 815,15.96,13.09
...,...,...,...,...,...,...
361,Social Science and Liberal Arts,Anti-Corruption,1,size: 1,0.01,100.00
362,Others,Chemicals & Fertilizers,1,size: 1,0.10,1.03
363,Others,Mail Management,1,size: 1,0.10,4.76
364,Finance,Public Policy,1,size: 1,0.07,20.00


In [7]:
test = "Governance, Law and Management"

In [11]:
sum_subject_count = df.loc[df["Subject"] == test]["Count"].sum()

In [16]:
df.loc[df["Subject"] == test, "p_Subject"] = df.loc[df["Subject"] == test]["Count"].apply(lambda x: round(((x / sum_subject_count) * 100), 2))

In [23]:
df.loc[df["Subject"] == test, "p_Subject"].sum()

99.97999999999999

In [5]:
df

,Subject,Experience,Count
0,Social Science and Liberal Arts,Land Revenue Mgmt & District Admn,1736
1,"Governance, Law and Management",Land Revenue Mgmt & District Admn,1227
2,"Engineering, Technology and Mathematics",Land Revenue Mgmt & District Admn,1104
3,Social Science and Liberal Arts,Personnel and General Administration,828
4,Natural Sciences,Land Revenue Mgmt & District Admn,815
...,...,...,...
361,Social Science and Liberal Arts,Anti-Corruption,1
362,Others,Chemicals & Fertilizers,1
363,Others,Mail Management,1
364,Finance,Public Policy,1


In [20]:
admin_categories = [
    "Land Revenue Mgmt & District Admn",
    "Personnel and General Administration",
    "Finance"
]
def filter_by_value(df, col, value, include_admin=True, number_of_rows=5, percentage=False):

    df = df.loc[df[col].str.contains(value)]
    df.sort_values(["Count"], ascending=False, inplace=True)

    if include_admin and col == "Subject":
        for category in admin_categories:
            df = df.loc[df["Experience"] != category]

    df.drop(columns=[col], inplace=True)
    df.set_index(df.columns[0], inplace=True)

    df = df.head(number_of_rows)

    if percentage:
        sum = df["Count"].sum()
        df = pd.DataFrame(df["Count"].apply(lambda x: (x / sum)))

    return df

In [21]:
filter_by_value(df, "Subject", "Governance, Law and Management", include_admin=False, number_of_rows=5, percentage=True)

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Count
Experience,
Land Revenue Mgmt & District Admn,0.390391
Personnel and General Administration,0.187082
Finance,0.176901
Industries,0.125040
Agriculture & Cooperation,0.120585


In [25]:
columns = df.columns

In [26]:
columns.get_loc("Subject")

0

In [27]:
def filter_by_value(df, col, value):

    df = df.loc[df[col].str.contains(value)]
    df = df.drop(columns=[col])
    df.set_index(df.columns[0], inplace=True)
    df.sort_values(["Count"], ascending=False, inplace=True)

    return df

In [28]:
df["Subject"].unique()[0]

'Social Science and Liberal Arts'

In [29]:
df.columns

Index(['Subject', 'Experience', 'Count'], dtype='object')

In [34]:
filter_by_value(df, "Experience", "Home").head(5)

,Count
Subject,
Social Science and Liberal Arts,396
"Governance, Law and Management",305
Natural Sciences,200
"Engineering, Technology and Mathematics",194
Languages and Fine Arts,184
